# 사용할 라이브러리 로딩

In [139]:
! pip install -U imbalanced-learn

In [137]:
import numpy as np # Numpy
import pandas as pd # Pandas
import matplotlib as mpl #Matplotlib 세팅용
import matplotlib.pyplot as plt # 시각화 도구
import seaborn as sns # 시각화 도구
from sklearn.model_selection import train_test_split # 데이터셋 분리
from sklearn.cluster import KMeans # 클러스터링
from sklearn.metrics import silhouette_score # 실루엣 점수
import xgboost as xgb # XGBoost
from xgboost import XGBClassifier  # XGBoostClassifier
from sklearn.model_selection import GridSearchCV # 그리드 서치
from sklearn.metrics import accuracy_score, precision_score # 평가 지표
from sklearn.metrics import recall_score, confusion_matrix, roc_auc_score, f1_score # 평가 지표
from imblearn.combine import *
from sklearn.ensemble import RandomForestClassifier # 랜덤 포레스트
from hyperopt import hp, STATUS_OK, fmin, tpe, Trials # 최적의 파람
from imblearn.combine import * # 복합 샘플링
from sklearn.model_selection import cross_val_score # 교차 스코어
import warnings # 경고문 제거용
from imblearn.combine import SMOTEENN, SMOTETomek # 복합샘플링
from sklearn.model_selection import KFold
from IPython.display import Image

from imblearn.over_sampling import *
%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

# 한글 폰트 설정
mpl.rc('font', family='D2Coding')
# 유니코드에서 음수 부호 설정
mpl.rc('axes', unicode_minus = False)

warnings.filterwarnings('ignore')
sns.set(font="D2Coding", rc={"axes.unicode_minus":False}, style='darkgrid')
plt.rc('figure', figsize=(10,8))

# 필요함수 만들기

In [3]:
# 스케일링을 위한 함수 생성
col = []
def data_scaled(df, col):
    for i in col:
        data_mean = df[i].mean()
        data_std = df[i].std()
        scaled = (df[i]-data_mean)/data_std
        df[i]=scaled
    return df

In [4]:
# 평가를 위한 함수 생성
def  get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
#     roc_auc = roc_auc_score(y_test, pred_proba)
    
    print('오차 행렬')
    print(confusion)
 
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, \
    재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

In [5]:
# 평가를 위한 함수 생성
def  get_multi_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
#     roc_auc = roc_auc_score(y_test, pred_proba)
    
    print('오차 행렬')
    print(confusion)
 
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, \
    재현율: {2:.4f}, F1: {3:.4f}'.format(accuracy, precision, recall, f1))

# 데이터 불러오기

- PassengerId는 'gggg_pp' 형태를 띔
- 같은 'gggg'는 그룹을 의미하며 pp는 그룹내에서의 번호를 의미함
- 이를 바탕으로 1차적으로 Cabin의 결측값을 채워줌
- 그후 Cabin을 문자/ 숫자/ 문자/ 문자+숫자 형태로 나눈 새로운 컬럼 4를 만들어 준 데이터 세트

- 개인
![nn](Individual.png)
<br>
- 그룹
![nn](Group1.png)
![nn](Group2.png)

In [6]:
spaceship = pd.read_excel('train_test_origin.xlsx')
print(spaceship.shape)

(12970, 18)


In [7]:
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin1,Cabin2,Combi,Cabin3,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,0.0,B,0.0,B0,P,B/0/P,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0
1,0002_01,Earth,0.0,F,0.0,F0,S,F/0/S,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0
2,0003_01,Europa,0.0,A,0.0,A0,S,A/0/S,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0
3,0003_02,Europa,0.0,A,0.0,A0,S,A/0/S,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0
4,0004_01,Earth,0.0,F,1.0,F1,S,F/1/S,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0


# 데이터 탐색

## 데이터 정보 확인

In [8]:
spaceship.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   12970 non-null  object 
 1   HomePlanet    12691 non-null  object 
 2   CryoSleep     12660 non-null  float64
 3   Cabin1        12804 non-null  object 
 4   Cabin2        12804 non-null  float64
 5   Combi         12804 non-null  object 
 6   Cabin3        12804 non-null  object 
 7   Cabin         12804 non-null  object 
 8   Destination   12704 non-null  object 
 9   Age           12700 non-null  float64
 10  VIP           12674 non-null  float64
 11  RoomService   12776 non-null  float64
 12  FoodCourt     12752 non-null  float64
 13  ShoppingMall  12760 non-null  float64
 14  Spa           12754 non-null  float64
 15  VRDeck        12766 non-null  float64
 16  Name          12676 non-null  object 
 17  Transported   8693 non-null   float64
dtypes: float64(10), object(8)


## 결측값 확인

In [9]:
spaceship.isnull().sum()

PassengerId        0
HomePlanet       279
CryoSleep        310
Cabin1           166
Cabin2           166
Combi            166
Cabin3           166
Cabin            166
Destination      266
Age              270
VIP              296
RoomService      194
FoodCourt        218
ShoppingMall     210
Spa              216
VRDeck           204
Name             294
Transported     4277
dtype: int64

- 결측값 처리가 필요한 피처들 : 
<br> HomePlanet(279), CryoSleep(310), Destination(266), Age(270), VIP(296), RoomService(194), FoodCourt(218), ShoppingMall(210), Spa(216), VRDeck(204)

# 전처리

## 불필요한 피처 제거 : PassengerId, Name, Combi, Cabin, Transported

In [10]:
spaceship.drop(['PassengerId', 'Name','Combi','Cabin','Transported'], inplace=True, axis=1)

In [11]:
spaceship.info() # 제거 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    12691 non-null  object 
 1   CryoSleep     12660 non-null  float64
 2   Cabin1        12804 non-null  object 
 3   Cabin2        12804 non-null  float64
 4   Cabin3        12804 non-null  object 
 5   Destination   12704 non-null  object 
 6   Age           12700 non-null  float64
 7   VIP           12674 non-null  float64
 8   RoomService   12776 non-null  float64
 9   FoodCourt     12752 non-null  float64
 10  ShoppingMall  12760 non-null  float64
 11  Spa           12754 non-null  float64
 12  VRDeck        12766 non-null  float64
dtypes: float64(9), object(4)
memory usage: 1.3+ MB


## 피처(CryoSleep, Cabin3, VIP) Dtype 변경 : object ▶ boolean

In [12]:
# Cabin3의 P(좌현)를 False으로 S(우현)를 True로 변경
spaceship['Cabin3'].replace({'P': True,'S': False}, inplace=True)

# boolean으로 변환
spaceship['CryoSleep'] = spaceship['CryoSleep'].astype(bool)
spaceship['VIP'] = spaceship['VIP'].astype(bool)
spaceship['Cabin3'] = spaceship['Cabin3'].astype(bool)

# 변환 확인
spaceship.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    12691 non-null  object 
 1   CryoSleep     12970 non-null  bool   
 2   Cabin1        12804 non-null  object 
 3   Cabin2        12804 non-null  float64
 4   Cabin3        12970 non-null  bool   
 5   Destination   12704 non-null  object 
 6   Age           12700 non-null  float64
 7   VIP           12970 non-null  bool   
 8   RoomService   12776 non-null  float64
 9   FoodCourt     12752 non-null  float64
 10  ShoppingMall  12760 non-null  float64
 11  Spa           12754 non-null  float64
 12  VRDeck        12766 non-null  float64
dtypes: bool(3), float64(7), object(3)
memory usage: 1.0+ MB


#  CryoSleep 결측값 처리

## 데이터 불러오기

In [13]:
cs_df = spaceship.iloc[:,:]
cs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    12691 non-null  object 
 1   CryoSleep     12970 non-null  bool   
 2   Cabin1        12804 non-null  object 
 3   Cabin2        12804 non-null  float64
 4   Cabin3        12970 non-null  bool   
 5   Destination   12704 non-null  object 
 6   Age           12700 non-null  float64
 7   VIP           12970 non-null  bool   
 8   RoomService   12776 non-null  float64
 9   FoodCourt     12752 non-null  float64
 10  ShoppingMall  12760 non-null  float64
 11  Spa           12754 non-null  float64
 12  VRDeck        12766 non-null  float64
dtypes: bool(3), float64(7), object(3)
memory usage: 1.0+ MB


## 전처리

In [14]:
cs_df.dropna(axis=0,inplace=True)
cs_df.shape

(11076, 13)

In [15]:
# 원-핫 인코딩 (cabin1, destination)

## HomePlanet
encoding = pd.get_dummies(cs_df.HomePlanet)
cs_df = cs_df.drop('HomePlanet', axis =1) # 기존 삭제
cs_df = cs_df.join(encoding) # 적용
## Cabin1
encoding = pd.get_dummies(cs_df.Cabin1)
cs_df = cs_df.drop('Cabin1', axis =1) # 기존 삭제
cs_df = cs_df.join(encoding) # 적용
## Destination
encoding = pd.get_dummies(cs_df.Destination)
cs_df = cs_df.drop('Destination', axis =1) # 기존 삭제
cs_df = cs_df.join(encoding) # 적용

In [16]:
cs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11076 entries, 0 to 12969
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CryoSleep      11076 non-null  bool   
 1   Cabin2         11076 non-null  float64
 2   Cabin3         11076 non-null  bool   
 3   Age            11076 non-null  float64
 4   VIP            11076 non-null  bool   
 5   RoomService    11076 non-null  float64
 6   FoodCourt      11076 non-null  float64
 7   ShoppingMall   11076 non-null  float64
 8   Spa            11076 non-null  float64
 9   VRDeck         11076 non-null  float64
 10  Earth          11076 non-null  uint8  
 11  Europa         11076 non-null  uint8  
 12  Mars           11076 non-null  uint8  
 13  A              11076 non-null  uint8  
 14  B              11076 non-null  uint8  
 15  C              11076 non-null  uint8  
 16  D              11076 non-null  uint8  
 17  E              11076 non-null  uint8  
 18  F     

In [17]:
# 스케일링

## 위에 정의된 스케일링 함수 호출
col = ['Cabin2', 'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
data_scaled(cs_df, col)

,CryoSleep,Cabin2,Cabin3,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,B,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,False,-1.171058,True,39.0,False,-0.334616,-0.282674,-0.285975,-0.271173,-0.260034,...,1,0,0,0,0,0,0,0,0,1
1,False,-1.171058,False,24.0,False,-0.166861,-0.277057,-0.244125,0.217775,-0.220858,...,0,0,0,0,1,0,0,0,0,1
2,False,-1.171058,False,58.0,True,-0.268437,1.949128,-0.285975,5.709312,-0.216406,...,0,0,0,0,0,0,0,0,0,1
3,False,-1.171058,False,33.0,False,-0.334616,0.518053,0.335083,2.693687,-0.088194,...,0,0,0,0,0,0,0,0,0,1
4,False,-1.169112,False,16.0,False,0.131712,-0.238987,-0.033199,0.232025,-0.258253,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12962,True,1.737845,False,43.0,False,-0.334616,-0.282674,-0.285975,-0.271173,-0.260034,...,0,0,0,0,0,1,0,0,0,1
12963,False,-0.630138,False,43.0,False,-0.262281,-0.282674,6.160647,-0.271173,-0.260034,...,0,0,1,0,0,0,0,0,0,1
12964,False,2.323517,False,40.0,False,-0.334616,0.257177,-0.285975,-0.268502,-0.260034,...,0,0,0,0,1,0,0,0,0,1
12965,True,1.739790,False,34.0,False,-0.334616,-0.282674,-0.285975,-0.271173,-0.260034,...,0,0,0,0,0,1,0,0,0,1


## 데이터와 타겟값 분리

In [18]:
cs_data = cs_df.drop('CryoSleep',axis=1)
cs_label = cs_df.CryoSleep

In [19]:
cs_data.head()

,Cabin2,Cabin3,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Earth,...,B,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,-1.171058,True,39.0,False,-0.334616,-0.282674,-0.285975,-0.271173,-0.260034,0,...,1,0,0,0,0,0,0,0,0,1
1,-1.171058,False,24.0,False,-0.166861,-0.277057,-0.244125,0.217775,-0.220858,1,...,0,0,0,0,1,0,0,0,0,1
2,-1.171058,False,58.0,True,-0.268437,1.949128,-0.285975,5.709312,-0.216406,0,...,0,0,0,0,0,0,0,0,0,1
3,-1.171058,False,33.0,False,-0.334616,0.518053,0.335083,2.693687,-0.088194,0,...,0,0,0,0,0,0,0,0,0,1
4,-1.169112,False,16.0,False,0.131712,-0.238987,-0.033199,0.232025,-0.258253,1,...,0,0,0,0,1,0,0,0,0,1


## train, test 데이터세트 분리

In [20]:
X_train, X_test, y_train, y_test = train_test_split(cs_data,cs_label, random_state=109)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train)

## 모델링_XGBClassifier

### 그리드 서치

In [21]:
# 그리드 서치를 이용한 최적의 파라미터 찾기

xgbo = xgb.XGBClassifier()

params = {
    'max_depth':[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
                 19, 20, 21, 22, 23, 24, 25, None],
    'learning_rate':[0.01, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4],
    'gamma':[0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5],
    'random_state':[109]
}

gs = GridSearchCV(xgbo, param_grid = params, cv = 3, refit = True,
                  n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'gamma': [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5],
                         'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25,
                                           0.3, 0.35, 0.4],
                         'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                       14, 15, 16, 17, 18, 19, 20, 21, 22, 23,
                                       24, 25, None],
                         'random_state': [109]})

In [22]:
# 평가 결과
model = gs.best_estimator_
print(model.score(X_train, y_train))
print(model.score(X_test, y_test))
print(model.score(X_val, y_val))

0.9457463884430176
0.939689418562658
0.9383726528647087


In [23]:
# 최적의 파라미터값
print(gs.best_params_)

{'gamma': 1.5, 'learning_rate': 0.3, 'max_depth': 4, 'random_state': 109}


In [24]:
# 그리드서치를 통해 얻어진 파라미터를 적용시킨 XGBClassifier 모델로 학습
xgbo_gs = xgb.XGBClassifier(gamma=5, learning_rate=0.4, max_depth=6, random_state=109)
xgbo_gs.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=5, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.4, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=109, ...)

In [25]:
# 앞서 만든 평가 함수(사용자 지정 함수)를 위해 필요한 변수들
train_pred = xgbo_gs.predict(X_train)
train_proba = xgbo_gs.predict_proba(X_train)

test_pred = xgbo_gs.predict(X_test)
test_proba = xgbo_gs.predict_proba(X_test)

val_pred = xgbo_gs.predict(X_val)
val_proba = xgbo_gs.predict_proba(X_val)

In [26]:
print('훈련셋 평가 지표\n')
get_clf_eval(y_train, train_pred, train_proba)
print('\n\n 테스트셋 평가 지표\n')
get_clf_eval(y_test, test_pred, test_proba)
print('\n\n 검증셋 평가 지표\n')
get_clf_eval(y_val, val_pred, val_proba)

훈련셋 평가 지표

오차 행렬
[[3566  209]
 [ 146 2309]]
정확도: 0.9430, 정밀도: 0.9170,     재현율: 0.9405, F1: 0.9286


 테스트셋 평가 지표

오차 행렬
[[1590   96]
 [  67 1016]]
정확도: 0.9411, 정밀도: 0.9137,     재현율: 0.9381, F1: 0.9257


 검증셋 평가 지표

오차 행렬
[[1193   65]
 [  55  764]]
정확도: 0.9422, 정밀도: 0.9216,     재현율: 0.9328, F1: 0.9272


### HyperOPT

In [27]:
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 15, 1),
                   'min_child_weight': hp.quniform('min_child_weight', 1, 20, 1),
                   'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 0.95),
                   'learning_rate': hp.uniform('learning_rate', 0.01, 0.4),
                   'gamma': hp.uniform('gamma', 0, 4)}

In [28]:
# fmin()에서 호출 시 search_space 값으로 XGBClassifier 교차 검증 학습 후 -1 * roc_auc 평균 값을 반환
def bin_objective_func(search_space):
    xgb_clf = xgb.XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            colsample_bytree=search_space['colsample_bytree'],
                            learning_rate=search_space['learning_rate'],
                            gamma=search_space['gamma'])
    
    # 3개 k-fold 방식으로 평가된 roc_auc 지표를 담는 list
    roc_auc_list = []
    
    # 3개 k-fold 방식 적용
    kf = KFold(n_splits=3)
    
    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train):
        # kf.split(X_train)으로 추출된 학습과 검증 index 값으로 학습과 검증 데이터 세트 분리
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]
        
        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행
        xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric="auc",
                   eval_set=[(X_tr, y_tr), (X_val, y_val)])
        
        # 1로 예측한 확률값 추출 후 roc auc 계산하고 평균 roc auc 계산을 위해 list에 결과값 담음.
        score = roc_auc_score(y_val, xgb_clf.predict_proba(X_val)[:,1])
        roc_auc_list.append(score)
        
    # 3개 k-fold로 계산된 roc_auc 값의 평균값을 반환하되,
    # HyperOPT는 목적함수의 최솟값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환
    return -1*np.mean(roc_auc_list)

In [29]:
# HyperOPT를 이용한 최적의 파라미터 찾기

trials = Trials()

# fmin() 함수를 호출. max_evals 지정된 횟수만큼 반복 후
# 목적함수의 최솟값을 가지는 최적 입력값 추출
best = fmin(fn=bin_objective_func,
           space=xgb_search_space,
           algo=tpe.suggest,
           max_evals=50, # 최대 반복 횟수를 지정
           trials=trials, rstate=np.random.default_rng(seed=109))

print('best:', best)

[0]	validation_0-auc:0.95288	validation_1-auc:0.94657                                                                  
[1]	validation_0-auc:0.95631	validation_1-auc:0.94681                                                                  
[2]	validation_0-auc:0.97439	validation_1-auc:0.97006                                                                  
[3]	validation_0-auc:0.97489	validation_1-auc:0.97030                                                                  
[4]	validation_0-auc:0.97944	validation_1-auc:0.97099                                                                  
[5]	validation_0-auc:0.98087	validation_1-auc:0.97191                                                                  
[6]	validation_0-auc:0.98332	validation_1-auc:0.97290                                                                  
[7]	validation_0-auc:0.98369	validation_1-auc:0.97347                                                                  
[8]	validation_0-auc:0.98470	validation_

[19]	validation_0-auc:0.98935	validation_1-auc:0.97469                                                                 
[20]	validation_0-auc:0.98965	validation_1-auc:0.97445                                                                 
[21]	validation_0-auc:0.98968	validation_1-auc:0.97442                                                                 
[22]	validation_0-auc:0.98968	validation_1-auc:0.97442                                                                 
[23]	validation_0-auc:0.98998	validation_1-auc:0.97417                                                                 
[24]	validation_0-auc:0.98998	validation_1-auc:0.97417                                                                 
[25]	validation_0-auc:0.98998	validation_1-auc:0.97417                                                                 
[26]	validation_0-auc:0.99005	validation_1-auc:0.97396                                                                 
[27]	validation_0-auc:0.99068	validation

[16]	validation_0-auc:0.97534	validation_1-auc:0.97354                                                                 
[17]	validation_0-auc:0.97666	validation_1-auc:0.97372                                                                 
[18]	validation_0-auc:0.97669	validation_1-auc:0.97394                                                                 
[19]	validation_0-auc:0.97663	validation_1-auc:0.97368                                                                 
[20]	validation_0-auc:0.97684	validation_1-auc:0.97400                                                                 
[21]	validation_0-auc:0.97794	validation_1-auc:0.97452                                                                 
[22]	validation_0-auc:0.97812	validation_1-auc:0.97408                                                                 
[23]	validation_0-auc:0.97871	validation_1-auc:0.97413                                                                 
[24]	validation_0-auc:0.97912	validation

[14]	validation_0-auc:0.97425	validation_1-auc:0.97361                                                                 
[15]	validation_0-auc:0.97500	validation_1-auc:0.97398                                                                 
[16]	validation_0-auc:0.97541	validation_1-auc:0.97410                                                                 
[17]	validation_0-auc:0.97682	validation_1-auc:0.97430                                                                 
[18]	validation_0-auc:0.97729	validation_1-auc:0.97437                                                                 
[19]	validation_0-auc:0.97720	validation_1-auc:0.97485                                                                 
[20]	validation_0-auc:0.97770	validation_1-auc:0.97468                                                                 
[21]	validation_0-auc:0.97842	validation_1-auc:0.97480                                                                 
[22]	validation_0-auc:0.97835	validation

[18]	validation_0-auc:0.97723	validation_1-auc:0.96891                                                                 
[19]	validation_0-auc:0.97765	validation_1-auc:0.96804                                                                 
[20]	validation_0-auc:0.97810	validation_1-auc:0.96812                                                                 
[21]	validation_0-auc:0.97806	validation_1-auc:0.96816                                                                 
[22]	validation_0-auc:0.97802	validation_1-auc:0.96796                                                                 
[23]	validation_0-auc:0.97884	validation_1-auc:0.96857                                                                 
[24]	validation_0-auc:0.97942	validation_1-auc:0.96802                                                                 
[25]	validation_0-auc:0.98014	validation_1-auc:0.96795                                                                 
[26]	validation_0-auc:0.98036	validation

...

[11]	validation_0-auc:0.99019	validation_1-auc:0.96638                                                                 
[12]	validation_0-auc:0.99023	validation_1-auc:0.96700                                                                 
[13]	validation_0-auc:0.99078	validation_1-auc:0.96726                                                                 
[14]	validation_0-auc:0.99116	validation_1-auc:0.96751                                                                 
[15]	validation_0-auc:0.99180	validation_1-auc:0.96741                                                                 
[16]	validation_0-auc:0.99218	validation_1-auc:0.96775                                                                 
[17]	validation_0-auc:0.99240	validation_1-auc:0.96838                                                                 
[18]	validation_0-auc:0.99257	validation_1-auc:0.96833                                                                 
[19]	validation_0-auc:0.99266	validation

[22]	validation_0-auc:0.98334	validation_1-auc:0.97237                                                                 
[23]	validation_0-auc:0.98340	validation_1-auc:0.97260                                                                 
[24]	validation_0-auc:0.98336	validation_1-auc:0.97262                                                                 
[25]	validation_0-auc:0.98372	validation_1-auc:0.97210                                                                 
[26]	validation_0-auc:0.98385	validation_1-auc:0.97215                                                                 
[27]	validation_0-auc:0.98400	validation_1-auc:0.97224                                                                 
[28]	validation_0-auc:0.98405	validation_1-auc:0.97223                                                                 
[29]	validation_0-auc:0.98436	validation_1-auc:0.97214                                                                 
[30]	validation_0-auc:0.98461	validation

[53]	validation_0-auc:0.98582	validation_1-auc:0.97552                                                                 
[54]	validation_0-auc:0.98581	validation_1-auc:0.97557                                                                 
[55]	validation_0-auc:0.98592	validation_1-auc:0.97551                                                                 
[56]	validation_0-auc:0.98608	validation_1-auc:0.97554                                                                 
[57]	validation_0-auc:0.98632	validation_1-auc:0.97555                                                                 
[58]	validation_0-auc:0.98636	validation_1-auc:0.97552                                                                 
[59]	validation_0-auc:0.98656	validation_1-auc:0.97559                                                                 
[60]	validation_0-auc:0.98652	validation_1-auc:0.97553                                                                 
[61]	validation_0-auc:0.98661	validation

[25]	validation_0-auc:0.98390	validation_1-auc:0.96727                                                                 
[26]	validation_0-auc:0.98384	validation_1-auc:0.96719                                                                 
[27]	validation_0-auc:0.98421	validation_1-auc:0.96753                                                                 
[28]	validation_0-auc:0.98426	validation_1-auc:0.96753                                                                 
[29]	validation_0-auc:0.98450	validation_1-auc:0.96745                                                                 
[30]	validation_0-auc:0.98466	validation_1-auc:0.96725                                                                 
[31]	validation_0-auc:0.98473	validation_1-auc:0.96750                                                                 
[32]	validation_0-auc:0.98487	validation_1-auc:0.96742                                                                 
[33]	validation_0-auc:0.98491	validation

[21]	validation_0-auc:0.97924	validation_1-auc:0.97428                                                                 
[22]	validation_0-auc:0.97944	validation_1-auc:0.97429                                                                 
[23]	validation_0-auc:0.97965	validation_1-auc:0.97400                                                                 
[24]	validation_0-auc:0.97976	validation_1-auc:0.97411                                                                 
[25]	validation_0-auc:0.97976	validation_1-auc:0.97411                                                                 
[26]	validation_0-auc:0.97976	validation_1-auc:0.97411                                                                 
[27]	validation_0-auc:0.97976	validation_1-auc:0.97411                                                                 
[28]	validation_0-auc:0.98011	validation_1-auc:0.97423                                                                 
[29]	validation_0-auc:0.98008	validation

In [30]:
# HyperOPT를 통해 얻어진 파라미터를 적용시킨 XGBClassifier 모델로 학습
xgbo = XGBClassifier(n_estimators=100, learning_rate=round(best['learning_rate'], 5), gamma=round(best['gamma'], 5),
                            max_depth=int(best['max_depth']), min_child_weight=int(best['min_child_weight']),
                            colsample_bytree=round(best['colsample_bytree'], 5), random_state = 109
                           )
xgbo.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.68423,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.17373, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.39916, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=12, max_leaves=0, min_child_weight=3, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=109, ...)

In [31]:
# 앞서 만든 평가 함수(사용자 지정 함수)를 위해 필요한 변수들
train_pred = xgbo.predict(X_train)
train_proba = xgbo.predict_proba(X_train)

test_pred = xgbo.predict(X_test)
test_proba = xgbo.predict_proba(X_test)

val_pred = xgbo.predict(X_val)
val_proba = xgbo.predict_proba(X_val)

In [32]:
print('훈련셋 평가 지표\n')
get_clf_eval(y_train, train_pred, train_proba)
print('\n\n 테스트셋 평가 지표\n')
get_clf_eval(y_test, test_pred, test_proba)
print('\n\n 검증셋 평가 지표\n')
get_clf_eval(y_val, val_pred, val_proba)

훈련셋 평가 지표

오차 행렬
[[3718   57]
 [  24 2431]]
정확도: 0.9870, 정밀도: 0.9771,     재현율: 0.9902, F1: 0.9836


 테스트셋 평가 지표

오차 행렬
[[1596   90]
 [ 113  970]]
정확도: 0.9267, 정밀도: 0.9151,     재현율: 0.8957, F1: 0.9053


 검증셋 평가 지표

오차 행렬
[[1201   57]
 [  73  746]]
정확도: 0.9374, 정밀도: 0.9290,     재현율: 0.9109, F1: 0.9199


## 결측값 채우기

In [33]:
c_na = pd.read_excel('na_slp.xlsx')

In [34]:
c_na.head()

,PassengerId,HomePlanet,CryoSleep,Cabin1,Cabin2,Combi,Cabin3,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0099_02,Earth,NaN,G,12.0,G12,P,G/12/P,TRAPPIST-1e,2.0,0.0,0.0,0.0,0.0,0.0,0.0,Thewis Connelson,1.0
1,0105_01,Earth,NaN,F,21.0,F21,P,F/21/P,TRAPPIST-1e,27.0,0.0,0.0,0.0,570.0,2.0,131.0,Carry Cleachrand,0.0
2,0110_02,Europa,NaN,B,5.0,B5,P,B/5/P,TRAPPIST-1e,40.0,0.0,0.0,331.0,0.0,0.0,1687.0,Aldeba Bootious,0.0
3,0115_01,Mars,NaN,F,24.0,F24,P,F/24/P,TRAPPIST-1e,26.0,0.0,0.0,0.0,0.0,0.0,0.0,Rohs Pead,1.0
4,0173_01,Earth,NaN,E,11.0,E11,S,E/11/S,TRAPPIST-1e,58.0,0.0,0.0,985.0,0.0,5.0,0.0,Hilip Grifford,1.0


In [35]:
c_na.drop(['PassengerId', 'Name', 'Transported', 'Combi', 'Cabin', 'CryoSleep'], inplace=True, axis=1)

In [36]:
c_na.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    305 non-null    object 
 1   Cabin1        306 non-null    object 
 2   Cabin2        306 non-null    float64
 3   Cabin3        306 non-null    object 
 4   Destination   304 non-null    object 
 5   Age           307 non-null    float64
 6   VIP           301 non-null    float64
 7   RoomService   307 non-null    float64
 8   FoodCourt     307 non-null    float64
 9   ShoppingMall  300 non-null    float64
 10  Spa           303 non-null    float64
 11  VRDeck        303 non-null    float64
dtypes: float64(8), object(4)
memory usage: 29.2+ KB


In [37]:
c_na['Cabin3'].replace({'P': 'True','S': 'False'}, inplace=True)

In [38]:
c_na['Cabin3'] = c_na.Cabin3.astype(bool)
c_na['VIP'] = c_na.VIP.astype(bool)
c_na.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    305 non-null    object 
 1   Cabin1        306 non-null    object 
 2   Cabin2        306 non-null    float64
 3   Cabin3        310 non-null    bool   
 4   Destination   304 non-null    object 
 5   Age           307 non-null    float64
 6   VIP           310 non-null    bool   
 7   RoomService   307 non-null    float64
 8   FoodCourt     307 non-null    float64
 9   ShoppingMall  300 non-null    float64
 10  Spa           303 non-null    float64
 11  VRDeck        303 non-null    float64
dtypes: bool(2), float64(7), object(3)
memory usage: 24.9+ KB


In [39]:
# object 타입의 데이터들 더미화
train_encoding = pd.get_dummies(c_na['HomePlanet'])
c_na=c_na.drop('HomePlanet',axis=1)
c_na = c_na.join(train_encoding)

train_encoding = pd.get_dummies(c_na['Cabin1'])
c_na=c_na.drop('Cabin1',axis=1)
c_na = c_na.join(train_encoding)

train_encoding = pd.get_dummies(c_na['Destination'])
c_na=c_na.drop('Destination',axis=1)
c_na = c_na.join(train_encoding)

In [40]:
c_na.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310 entries, 0 to 309
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Cabin2         306 non-null    float64
 1   Cabin3         310 non-null    bool   
 2   Age            307 non-null    float64
 3   VIP            310 non-null    bool   
 4   RoomService    307 non-null    float64
 5   FoodCourt      307 non-null    float64
 6   ShoppingMall   300 non-null    float64
 7   Spa            303 non-null    float64
 8   VRDeck         303 non-null    float64
 9   Earth          310 non-null    uint8  
 10  Europa         310 non-null    uint8  
 11  Mars           310 non-null    uint8  
 12  A              310 non-null    uint8  
 13  B              310 non-null    uint8  
 14  C              310 non-null    uint8  
 15  D              310 non-null    uint8  
 16  E              310 non-null    uint8  
 17  F              310 non-null    uint8  
 18  G         

In [41]:
c_na_d = c_na.dropna(axis=0)

In [42]:
col = ['Cabin2', 'Age', 'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
data_scaled(c_na, col)

,Cabin2,Cabin3,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Earth,...,B,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,-1.183897,True,-1.800699,False,-0.329359,-0.324266,-0.351021,-0.236142,-0.287000,1,...,0,0,0,0,0,1,0,0,0,1
1,-1.166462,True,-0.116726,False,-0.329359,-0.324266,0.975138,-0.234693,-0.156691,1,...,0,0,0,0,1,0,0,0,0,1
2,-1.197458,True,0.758940,False,-0.329359,-0.064159,-0.351021,-0.236142,1.391106,0,...,1,0,0,0,0,0,0,0,0,1
3,-1.160650,True,-0.184085,False,-0.329359,-0.324266,-0.351021,-0.236142,-0.287000,0,...,0,0,0,0,1,0,0,0,0,1
4,-1.185834,True,1.971401,False,-0.329359,0.449770,-0.351021,-0.232519,-0.287000,1,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,2.260524,True,-0.588239,False,-0.329359,-0.324266,-0.351021,-0.236142,-0.287000,0,...,0,0,0,0,1,0,0,1,0,0
306,-0.635657,True,-0.116726,False,-0.329359,1.390402,-0.334735,1.634807,-0.268100,0,...,0,1,0,0,0,0,0,0,0,1
307,NaN,True,0.826299,False,-0.329359,1.245810,-0.351021,0.505136,0.575429,0,...,0,0,0,0,0,0,0,0,0,1
308,2.233403,True,-0.992392,False,-0.329359,-0.324266,NaN,0.761649,-0.258153,1,...,0,0,0,0,1,0,0,0,0,1


In [43]:
c_na.head()

,Cabin2,Cabin3,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Earth,...,B,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,-1.183897,True,-1.800699,False,-0.329359,-0.324266,-0.351021,-0.236142,-0.287000,1,...,0,0,0,0,0,1,0,0,0,1
1,-1.166462,True,-0.116726,False,-0.329359,-0.324266,0.975138,-0.234693,-0.156691,1,...,0,0,0,0,1,0,0,0,0,1
2,-1.197458,True,0.758940,False,-0.329359,-0.064159,-0.351021,-0.236142,1.391106,0,...,1,0,0,0,0,0,0,0,0,1
3,-1.160650,True,-0.184085,False,-0.329359,-0.324266,-0.351021,-0.236142,-0.287000,0,...,0,0,0,0,1,0,0,0,0,1
4,-1.185834,True,1.971401,False,-0.329359,0.449770,-0.351021,-0.232519,-0.287000,1,...,0,0,0,1,0,0,0,0,0,1


### 그리드서치로 예측

In [44]:
c_pred = xgbo_gs.predict(c_na)

In [45]:
c_pred = pd.DataFrame(c_pred)

In [46]:
c_pred.value_counts()

0    310
dtype: int64

In [47]:
c_pred.to_csv('c_pred.csv')

### HyperOPT로 예측

In [48]:
c_pred_h = xgbo.predict(c_na)

In [49]:
c_pred_h = pd.DataFrame(c_pred_h)

In [50]:
c_pred_h.value_counts()

0    310
dtype: int64

# HomePlanet 결측값 처리

## 데이터 불러오기

In [51]:
# HomePlanet에 대한 데이터프레임 생성
hp_df = spaceship.iloc[:,:]

# 앞서 제거한 피처 외 추가제거
hp_df = hp_df.drop('Age', axis=1)

In [52]:
hp_df.head()

,HomePlanet,CryoSleep,Cabin1,Cabin2,Cabin3,Destination,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Europa,False,B,0.0,True,TRAPPIST-1e,False,0.0,0.0,0.0,0.0,0.0
1,Earth,False,F,0.0,False,TRAPPIST-1e,False,109.0,9.0,25.0,549.0,44.0
2,Europa,False,A,0.0,False,TRAPPIST-1e,True,43.0,3576.0,0.0,6715.0,49.0
3,Europa,False,A,0.0,False,TRAPPIST-1e,False,0.0,1283.0,371.0,3329.0,193.0
4,Earth,False,F,1.0,False,TRAPPIST-1e,False,303.0,70.0,151.0,565.0,2.0


## 전처리

In [53]:
# 결측값이 있는 행 삭제 
hp_df.dropna(inplace=True)
hp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11076 entries, 0 to 12969
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    11076 non-null  object 
 1   CryoSleep     11076 non-null  bool   
 2   Cabin1        11076 non-null  object 
 3   Cabin2        11076 non-null  float64
 4   Cabin3        11076 non-null  bool   
 5   Destination   11076 non-null  object 
 6   VIP           11076 non-null  bool   
 7   RoomService   11076 non-null  float64
 8   FoodCourt     11076 non-null  float64
 9   ShoppingMall  11076 non-null  float64
 10  Spa           11076 non-null  float64
 11  VRDeck        11076 non-null  float64
dtypes: bool(3), float64(6), object(3)
memory usage: 897.8+ KB


In [54]:
# homeplanet이 타깃으로 할거라서 홈플레닛 값을 0,1,2로 치환하여 사용
hp_df.HomePlanet = hp_df['HomePlanet'].map({'Earth':0, 'Europa':1, 'Mars':2})

# 변경 확인
hp_df.HomePlanet.unique()

array([1, 0, 2], dtype=int64)

In [55]:
# 원-핫 인코딩 (cabin1, destination)

## Cabin1
encoding = pd.get_dummies(hp_df.Cabin1)
hp_df = hp_df.drop('Cabin1', axis =1) # 기존 삭제
hp_df = hp_df.join(encoding) # 적용
## Destination
encoding = pd.get_dummies(hp_df.Destination)
hp_df = hp_df.drop('Destination', axis =1) # 기존 삭제
hp_df = hp_df.join(encoding) # 적용

In [56]:
hp_df.head()

,HomePlanet,CryoSleep,Cabin2,Cabin3,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,B,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,1,False,0.0,True,False,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,1
1,0,False,0.0,False,False,109.0,9.0,25.0,549.0,44.0,...,0,0,0,0,1,0,0,0,0,1
2,1,False,0.0,False,True,43.0,3576.0,0.0,6715.0,49.0,...,0,0,0,0,0,0,0,0,0,1
3,1,False,0.0,False,False,0.0,1283.0,371.0,3329.0,193.0,...,0,0,0,0,0,0,0,0,0,1
4,0,False,1.0,False,False,303.0,70.0,151.0,565.0,2.0,...,0,0,0,0,1,0,0,0,0,1


In [57]:
hp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11076 entries, 0 to 12969
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   HomePlanet     11076 non-null  int64  
 1   CryoSleep      11076 non-null  bool   
 2   Cabin2         11076 non-null  float64
 3   Cabin3         11076 non-null  bool   
 4   VIP            11076 non-null  bool   
 5   RoomService    11076 non-null  float64
 6   FoodCourt      11076 non-null  float64
 7   ShoppingMall   11076 non-null  float64
 8   Spa            11076 non-null  float64
 9   VRDeck         11076 non-null  float64
 10  A              11076 non-null  uint8  
 11  B              11076 non-null  uint8  
 12  C              11076 non-null  uint8  
 13  D              11076 non-null  uint8  
 14  E              11076 non-null  uint8  
 15  F              11076 non-null  uint8  
 16  G              11076 non-null  uint8  
 17  T              11076 non-null  uint8  
 18  55 Can

In [58]:
# 스케일링

## 위에 정의된 스케일링 함수 호출
col = ['Cabin2', 'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
data_scaled(hp_df, col)

,HomePlanet,CryoSleep,Cabin2,Cabin3,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,B,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,1,False,-1.171058,True,False,-0.334616,-0.282674,-0.285975,-0.271173,-0.260034,...,1,0,0,0,0,0,0,0,0,1
1,0,False,-1.171058,False,False,-0.166861,-0.277057,-0.244125,0.217775,-0.220858,...,0,0,0,0,1,0,0,0,0,1
2,1,False,-1.171058,False,True,-0.268437,1.949128,-0.285975,5.709312,-0.216406,...,0,0,0,0,0,0,0,0,0,1
3,1,False,-1.171058,False,False,-0.334616,0.518053,0.335083,2.693687,-0.088194,...,0,0,0,0,0,0,0,0,0,1
4,0,False,-1.169112,False,False,0.131712,-0.238987,-0.033199,0.232025,-0.258253,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12962,0,True,1.737845,False,False,-0.334616,-0.282674,-0.285975,-0.271173,-0.260034,...,0,0,0,0,0,1,0,0,0,1
12963,2,False,-0.630138,False,False,-0.262281,-0.282674,6.160647,-0.271173,-0.260034,...,0,0,1,0,0,0,0,0,0,1
12964,0,False,2.323517,False,False,-0.334616,0.257177,-0.285975,-0.268502,-0.260034,...,0,0,0,0,1,0,0,0,0,1
12965,0,True,1.739790,False,False,-0.334616,-0.282674,-0.285975,-0.271173,-0.260034,...,0,0,0,0,0,1,0,0,0,1


## 데이터와 타겟값 분리

In [59]:
hp_data = hp_df.iloc[:,1:]
hp_label = hp_df.iloc[:,0]

In [60]:
hp_data.head()

,CryoSleep,Cabin2,Cabin3,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,A,B,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,False,-1.171058,True,False,-0.334616,-0.282674,-0.285975,-0.271173,-0.260034,0,1,0,0,0,0,0,0,0,0,1
1,False,-1.171058,False,False,-0.166861,-0.277057,-0.244125,0.217775,-0.220858,0,0,0,0,0,1,0,0,0,0,1
2,False,-1.171058,False,True,-0.268437,1.949128,-0.285975,5.709312,-0.216406,1,0,0,0,0,0,0,0,0,0,1
3,False,-1.171058,False,False,-0.334616,0.518053,0.335083,2.693687,-0.088194,1,0,0,0,0,0,0,0,0,0,1
4,False,-1.169112,False,False,0.131712,-0.238987,-0.033199,0.232025,-0.258253,0,0,0,0,0,1,0,0,0,0,1


In [61]:
hp_label

0        1
1        0
2        1
3        1
4        0
        ..
12962    0
12963    2
12964    0
12965    0
12969    0
Name: HomePlanet, Length: 11076, dtype: int64

## train, test 데이터세트 분리

In [62]:
X_train, X_test, y_train, y_test = train_test_split(hp_data,hp_label, random_state =109) 

## 모델링_랜덤포레스트

In [63]:
rf_clf = RandomForestClassifier()
grid = {
    'n_estimators': [50,90,100,150,200, 250],
    'max_depth': [3,5,7,9,13,15],
    'min_samples_leaf':[3,5,7,9,13,15],
    'min_samples_split': [3,5,7,9,13,15]}

clf_grid = GridSearchCV(rf_clf, param_grid = grid, scoring='accuracy', verbose=1, cv= 5, n_jobs=-1)

clf_grid.fit(X_train, y_train)


Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [3, 5, 7, 9, 13, 15],
                         'min_samples_leaf': [3, 5, 7, 9, 13, 15],
                         'min_samples_split': [3, 5, 7, 9, 13, 15],
                         'n_estimators': [50, 90, 100, 150, 200, 250]},
             scoring='accuracy', verbose=1)

In [64]:
print('최고 평균 정확도 : {}'.format(clf_grid.best_score_))
print('최고 파라미터: {}', clf_grid.best_params_)

최고 평균 정확도 : 0.9375231744610375
최고 파라미터: {} {'max_depth': 15, 'min_samples_leaf': 3, 'min_samples_split': 5, 'n_estimators': 250}


In [65]:
# 최적의 파라미터를 이용한 모델링
clf_model = RandomForestClassifier(n_estimators=150, max_depth=15, 
                                   min_samples_leaf=3, min_samples_split=5, random_state=109)

# fitting
clf_model.fit(X_train, y_train)

# pred
tr_clf_pred = clf_model.predict(X_train)
tr_clf_proba = clf_model.predict_proba(X_train)
clf_pred = clf_model.predict(X_test)
clf_proba = clf_model.predict_proba(X_test)

In [66]:
print('훈련셋 평가 지표\n')
get_multi_clf_eval(y_train, tr_clf_pred, tr_clf_proba)
print('\n\n 테스트셋 평가 지표\n')
get_multi_clf_eval(y_test, clf_pred, clf_proba)

훈련셋 평가 지표

오차 행렬
[[4367    3   86]
 [   8 2048   35]
 [ 126   34 1600]]
정확도: 0.9648, 정밀도: 0.9648,     재현율: 0.9648, F1: 0.9648


 테스트셋 평가 지표

오차 행렬
[[1436    1   40]
 [   4  695   11]
 [  89   13  480]]
정확도: 0.9429, 정밀도: 0.9429,     재현율: 0.9429, F1: 0.9429


In [67]:
# Mars의 예측 재현율이 다소 낮은걸 확인
510/(78+16+510)

0.8443708609271523

## 모델링_XGBoost

In [68]:
# hyperopt 적용을 위한 설정 값 담기
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 20, 1),
                    'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),
                    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
                    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
               }

In [69]:
# 실행을 위한 함수 정의
def objective_func(search_space):
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            learning_rate=search_space['learning_rate'],
                            colsample_bytree=search_space['colsample_bytree'], 
                            eval_metric='logloss')
    accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3)
    return {'loss':-1 * np.mean(accuracy), 'status': STATUS_OK}


In [70]:
# 최적의 파라미터 찾기
trial_val = Trials()
best = fmin(fn=objective_func, # 정의한 실행 함수
            space=xgb_search_space, # 하이퍼 오피티 설정값 정의한거
            algo=tpe.suggest,
            max_evals=50,
            trials=trial_val, rstate=np.random.default_rng(seed=9))
print('best:', best)

100%|███████████████████████████████████████████████| 50/50 [00:50<00:00,  1.01s/trial, best loss: -0.9428193090164921]
best: {'colsample_bytree': 0.7131644932071765, 'learning_rate': 0.16071485807351799, 'max_depth': 5.0, 'min_child_weight': 2.0}


In [71]:
# 모델링
xgb_model = XGBClassifier(n_estimators=100, learning_rate=round(best['learning_rate'], 5), 
                            max_depth=int(best['max_depth']), min_child_weight=int(best['min_child_weight']),
                            colsample_bytree=round(best['colsample_bytree'], 5)
                           )
xgb_model.fit(X_train, y_train)

tr_pred = xgb_model.predict(X_train)
tr_proba = xgb_model.predict_proba(X_train)
t_pred = xgb_model.predict(X_test)
t_proba = xgb_model.predict(X_test)

In [72]:
print('훈련셋 평가 지표\n')
get_multi_clf_eval(y_train, tr_pred, tr_proba)
print('\n\n 테스트셋 평가 지표\n')
get_multi_clf_eval(y_test, t_pred, t_proba)

훈련셋 평가 지표

오차 행렬
[[4349    1  106]
 [   2 2057   32]
 [  91   33 1636]]
정확도: 0.9681, 정밀도: 0.9681,     재현율: 0.9681, F1: 0.9681


 테스트셋 평가 지표

오차 행렬
[[1416    1   60]
 [   2  690   18]
 [  68   13  501]]
정확도: 0.9415, 정밀도: 0.9415,     재현율: 0.9415, F1: 0.9415


In [73]:
# Mars의 정확도
529/(63+14+527)*100

87.58278145695364

## 복합 샘플링
- 샘플의 편향이 심해서 Mars의 확률이 낮은거 같아서 복합 샘플링을 실행

In [74]:
# 샘플 편향
np.unique(hp_label, return_counts = True)

(array([0, 1, 2], dtype=int64), array([5933, 2801, 2342], dtype=int64))

In [75]:
# 복합 샘플링
X_samp, y_samp = SMOTEENN(random_state=109).fit_resample(hp_data, hp_label)

In [76]:
# 샘플링 완료
np.unique(y_samp, return_counts = True)

(array([0, 1, 2], dtype=int64), array([5111, 5614, 5031], dtype=int64))

## 샘플링 데이터세트 분리

In [77]:
X_train, X_test, y_train, y_test = train_test_split( X_samp, y_samp, random_state = 109 )

## 모델링_XGBClassifier & hyperopt

In [78]:
# 실행을 위한 함수 정의
def objective_func(search_space):
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            learning_rate=search_space['learning_rate'],
                            colsample_bytree=search_space['colsample_bytree'], 
                            eval_metric='logloss')
    accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3)
    return {'loss':-1 * np.mean(accuracy), 'status': STATUS_OK}

In [79]:
# 최적의 파라미터 찾기
trial_val = Trials()
best = fmin(fn=objective_func,
            space=xgb_search_space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trial_val, rstate=np.random.default_rng(seed=9))
print('best:', best)

100%|███████████████████████████████████████████████| 50/50 [01:07<00:00,  1.35s/trial, best loss: -0.9906913768299906]
best: {'colsample_bytree': 0.8052776945627771, 'learning_rate': 0.19910199851014435, 'max_depth': 12.0, 'min_child_weight': 1.0}


In [80]:
# 모델링
xgb_model = XGBClassifier(n_estimators=100, learning_rate=round(best['learning_rate'], 5), 
                            max_depth=int(best['max_depth']), min_child_weight=int(best['min_child_weight']),
                            colsample_bytree=round(best['colsample_bytree'], 5)
                           )
xgb_model.fit(X_train, y_train)

tr_pred = xgb_model.predict(X_train)
tr_proba = xgb_model.predict_proba(X_train)
t_pred = xgb_model.predict(X_test)
t_proba = xgb_model.predict(X_test)

In [81]:
print('훈련셋 평가 지표\n')
get_multi_clf_eval(y_train, tr_pred, tr_proba)
print('\n\n 테스트셋 평가 지표\n')
get_multi_clf_eval(y_test, t_pred, t_proba)

훈련셋 평가 지표

오차 행렬
[[3811    0    0]
 [   0 4229    0]
 [   0    0 3777]]
정확도: 1.0000, 정밀도: 1.0000,     재현율: 1.0000, F1: 1.0000


 테스트셋 평가 지표

오차 행렬
[[1282    2   16]
 [   1 1381    3]
 [  13    6 1235]]
정확도: 0.9896, 정밀도: 0.9896,     재현율: 0.9896, F1: 0.9896


In [82]:
# Mars의 정확도
1270/(5+1+1270)*100

99.52978056426332

## 결측값 채우기

In [83]:
# 이전과 동일한 절차

# 결측치 파일 불러오기
df = pd.read_excel('hp_na.xlsx')
df.head()
data = df.drop(['Transported', 'Name', 'Age', 'Cabin','Combi'], axis=1)
# Cabin1
encode = pd.get_dummies(data['Cabin1'])
data.drop('Cabin1', axis=1)
data = data.join(encode)
# Destination
encode = pd.get_dummies(data['Destination'])
data.drop(['Destination'], axis=1)
data = data.join(encode)
data['Cabin3'] = data['Cabin3'].map({'P':0, 'S':1})
data['HomePlanet'] = data['HomePlanet'].map({'Earth':0, 'Europa':1, 'Mars':2})
data_scaled(data, col)
dt = data.drop(['PassengerId', 'HomePlanet','Destination', 'Cabin1'], axis = 1)
label = data['HomePlanet']
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279 entries, 0 to 278
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CryoSleep      279 non-null    bool   
 1   Cabin2         274 non-null    float64
 2   Cabin3         274 non-null    float64
 3   VIP            276 non-null    float64
 4   RoomService    275 non-null    float64
 5   FoodCourt      276 non-null    float64
 6   ShoppingMall   275 non-null    float64
 7   Spa            272 non-null    float64
 8   VRDeck         277 non-null    float64
 9   A              279 non-null    uint8  
 10  B              279 non-null    uint8  
 11  C              279 non-null    uint8  
 12  D              279 non-null    uint8  
 13  E              279 non-null    uint8  
 14  F              279 non-null    uint8  
 15  G              279 non-null    uint8  
 16  T              279 non-null    uint8  
 17  55 Cancri e    279 non-null    uint8  
 18  PSO J318.5

In [84]:
df['predict_hp']= xgb_model.predict(dt)
df['predict_hp'] = df['predict_hp'].map({0:'Earth', 1:'Europa', 2:'Mars'})
df # 샘플링 하지 않고 처리한 것

,PassengerId,HomePlanet,CryoSleep,Cabin1,Cabin2,Combi,Cabin3,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,predict_hp
0,0064_02,NaN,True,E,3.0,E3,S,E/3/S,TRAPPIST-1e,33.0,0.0,0.0,0.0,0.0,0.0,0.0,Colatz Keen,1.0,Europa
1,0119_01,NaN,False,A,0.0,A0,P,A/0/P,TRAPPIST-1e,39.0,0.0,0.0,2344.0,0.0,65.0,6898.0,Batan Coning,0.0,Europa
2,0210_01,NaN,True,D,6.0,D6,P,D/6/P,55 Cancri e,24.0,0.0,0.0,0.0,0.0,0.0,0.0,Arraid Inicont,1.0,Europa
3,0242_01,NaN,False,F,46.0,F46,S,F/46/S,TRAPPIST-1e,18.0,0.0,313.0,1.0,691.0,283.0,0.0,Almone St챕,0.0,Mars
4,0251_01,NaN,True,C,11.0,C11,S,C/11/S,55 Cancri e,54.0,0.0,0.0,0.0,0.0,0.0,0.0,Diphah Amsive,1.0,Europa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,8621_01,NaN,False,E,552.0,E552,P,E/552/P,TRAPPIST-1e,19.0,0.0,4.0,0.0,1604.0,0.0,0.0,Vanley Simmonders,NaN,Mars
275,8678_01,NaN,True,G,1399.0,G1399,S,G/1399/S,55 Cancri e,9.0,0.0,0.0,0.0,0.0,0.0,0.0,Eilan Kellson,NaN,Earth
276,8775_01,NaN,True,D,275.0,D275,P,D/275/P,TRAPPIST-1e,40.0,0.0,0.0,0.0,0.0,0.0,0.0,Raston Maltorted,NaN,Europa
277,9025_01,NaN,False,G,1454.0,G1454,S,G/1454/S,TRAPPIST-1e,42.0,0.0,0.0,0.0,28.0,726.0,0.0,Ale Whitersone,NaN,Earth


In [85]:
# 예측값 (샘플링 후)
df['sampling_pred_hp'] = xgb_model.predict(dt)
# 원본 형태로 되돌리기
df['sampling_pred_hp'] = df['sampling_pred_hp'].map({0:'Earth', 1:'Europa', 2:'Mars'})

In [86]:
np.unique(df['sampling_pred_hp'], return_counts = True)

(array(['Earth', 'Europa', 'Mars'], dtype=object),
 array([136,  71,  72], dtype=int64))

In [87]:
#엑셀로 저장하기
df.to_excel('hp_fill(sampling).xlsx')

# Destination 결측값 처리

## 데이터 불러오기

In [88]:
dst_df = spaceship.iloc[:,:]
dst_df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11076 entries, 0 to 12969
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    11076 non-null  object 
 1   CryoSleep     11076 non-null  bool   
 2   Cabin1        11076 non-null  object 
 3   Cabin2        11076 non-null  float64
 4   Cabin3        11076 non-null  bool   
 5   Destination   11076 non-null  object 
 6   Age           11076 non-null  float64
 7   VIP           11076 non-null  bool   
 8   RoomService   11076 non-null  float64
 9   FoodCourt     11076 non-null  float64
 10  ShoppingMall  11076 non-null  float64
 11  Spa           11076 non-null  float64
 12  VRDeck        11076 non-null  float64
dtypes: bool(3), float64(7), object(3)
memory usage: 1.2+ MB


## 전처리

In [89]:
dst_df.dropna(axis=0,inplace=True)
dst_df.shape

(11076, 13)

In [90]:
# 원-핫 인코딩 (cabin1, destination)

## HomePlanet
encoding = pd.get_dummies(dst_df.HomePlanet)
dst_df = dst_df.drop('HomePlanet', axis =1) # 기존 삭제
dst_df = dst_df.join(encoding) # 적용
## Cabin1
encoding = pd.get_dummies(dst_df.Cabin1)
dst_df = dst_df.drop('Cabin1', axis =1) # 기존 삭제
dst_df = dst_df.join(encoding) # 적용

dst_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11076 entries, 0 to 12969
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CryoSleep     11076 non-null  bool   
 1   Cabin2        11076 non-null  float64
 2   Cabin3        11076 non-null  bool   
 3   Destination   11076 non-null  object 
 4   Age           11076 non-null  float64
 5   VIP           11076 non-null  bool   
 6   RoomService   11076 non-null  float64
 7   FoodCourt     11076 non-null  float64
 8   ShoppingMall  11076 non-null  float64
 9   Spa           11076 non-null  float64
 10  VRDeck        11076 non-null  float64
 11  Earth         11076 non-null  uint8  
 12  Europa        11076 non-null  uint8  
 13  Mars          11076 non-null  uint8  
 14  A             11076 non-null  uint8  
 15  B             11076 non-null  uint8  
 16  C             11076 non-null  uint8  
 17  D             11076 non-null  uint8  
 18  E             11076 non-nu

In [91]:
# 스케일링

## 위에 정의된 스케일링 함수 호출
col = ['Cabin2', 'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
data_scaled(dst_df, col)

,CryoSleep,Cabin2,Cabin3,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,...,Europa,Mars,A,B,C,D,E,F,G,T
0,False,-1.171058,True,TRAPPIST-1e,39.0,False,-0.334616,-0.282674,-0.285975,-0.271173,...,1,0,0,1,0,0,0,0,0,0
1,False,-1.171058,False,TRAPPIST-1e,24.0,False,-0.166861,-0.277057,-0.244125,0.217775,...,0,0,0,0,0,0,0,1,0,0
2,False,-1.171058,False,TRAPPIST-1e,58.0,True,-0.268437,1.949128,-0.285975,5.709312,...,1,0,1,0,0,0,0,0,0,0
3,False,-1.171058,False,TRAPPIST-1e,33.0,False,-0.334616,0.518053,0.335083,2.693687,...,1,0,1,0,0,0,0,0,0,0
4,False,-1.169112,False,TRAPPIST-1e,16.0,False,0.131712,-0.238987,-0.033199,0.232025,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12962,True,1.737845,False,TRAPPIST-1e,43.0,False,-0.334616,-0.282674,-0.285975,-0.271173,...,0,0,0,0,0,0,0,0,1,0
12963,False,-0.630138,False,TRAPPIST-1e,43.0,False,-0.262281,-0.282674,6.160647,-0.271173,...,0,1,0,0,0,1,0,0,0,0
12964,False,2.323517,False,TRAPPIST-1e,40.0,False,-0.334616,0.257177,-0.285975,-0.268502,...,0,0,0,0,0,0,0,1,0,0
12965,True,1.739790,False,TRAPPIST-1e,34.0,False,-0.334616,-0.282674,-0.285975,-0.271173,...,0,0,0,0,0,0,0,0,1,0


In [92]:
dst_df.Destination.unique()

array(['TRAPPIST-1e', 'PSO J318.5-22', '55 Cancri e'], dtype=object)

In [93]:
# Object 타입을 숫자형으로 변경하기
# 'TRAPPIST-1e' = 0, 'PSO J318.5-22' = 1, '55 Cancri e' = 2
dst_df.Destination.replace({'TRAPPIST-1e':0, '55 Cancri e':1, 'PSO J318.5-22':2}, inplace=True)

In [94]:
dst_df['Destination'].value_counts()

0    7715
1    2325
2    1036
Name: Destination, dtype: int64

## 데이터와 타겟값 분리

In [95]:
dst_data = dst_df.drop('Destination', axis =1) 
dst_label = dst_df['Destination']

## 복합샘플링

In [96]:
# 복합 샘플링
X_sample, y_sample = SMOTEENN(random_state =109).fit_resample(dst_data, dst_label)

In [97]:
y_sample.value_counts() # 아직 편향이 좀 있지만, 나아졌네

2    5619
1    4456
0    2504
Name: Destination, dtype: int64

## train, test 데이터 세트 분리

In [98]:
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, random_state=109)

## 모델링_XGBClassifier

In [99]:
# hyperopt 적용을 위한 설정 값 담기
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 20, 1),
                    'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),
                    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
                    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 0.95)
               }

In [100]:
# 실행을 위한 함수 정의
def objective_func(search_space):
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            learning_rate=search_space['learning_rate'],
                            colsample_bytree=search_space['colsample_bytree'], 
                            eval_metric='logloss')
    accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3)
    return {'loss':-1 * np.mean(accuracy), 'status': STATUS_OK}

In [101]:
# 최적의 파라미터 찾기
trial_val = Trials()
best = fmin(fn=objective_func,
            space=xgb_search_space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trial_val, rstate=np.random.default_rng(seed=9))
print('best:', best)

100%|████████████████████████████████████████████████| 50/50 [01:52<00:00,  2.25s/trial, best loss: -0.885519342872284]
best: {'colsample_bytree': 0.7689868312949918, 'learning_rate': 0.15499060596585923, 'max_depth': 16.0, 'min_child_weight': 1.0}


In [102]:
# HyperOPT를 통해 얻어진 파라미터를 적용시킨 XGBClassifier 모델로 학습
xgb_model = XGBClassifier(n_estimators=100, learning_rate=round(best['learning_rate'], 5), 
                            max_depth=int(best['max_depth']), min_child_weight=int(best['min_child_weight']),
                            colsample_bytree=round(best['colsample_bytree'], 5)
                           )
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.76899,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.15499, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=16, max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto', ...)

In [103]:
# 앞서 만든 평가 함수(사용자 지정 함수)를 위해 필요한 변수들
tr_pred = xgb_model.predict(X_train)
tr_proba = xgb_model.predict_proba(X_train)

t_pred = xgb_model.predict(X_test)
t_proba = xgb_model.predict_proba(X_test)

In [104]:
print('훈련셋 평가 지표\n')
get_multi_clf_eval(y_train, tr_pred, tr_proba)
print('\n\n 테스트셋 평가 지표\n')
get_multi_clf_eval(y_test, t_pred, t_proba)

훈련셋 평가 지표

오차 행렬
[[1893    0    0]
 [   0 3332    2]
 [   0    0 4207]]
정확도: 0.9998, 정밀도: 0.9998,     재현율: 0.9998, F1: 0.9998


 테스트셋 평가 지표

오차 행렬
[[ 485   53   73]
 [  24 1034   64]
 [  39   18 1355]]
정확도: 0.9138, 정밀도: 0.9138,     재현율: 0.9138, F1: 0.9138


## 결측값 채우기

In [106]:
# 이전과 동일한 절차

# 결측치 파일 불러오기
df = pd.read_excel('dest_na.xlsx')
df.head()
data = df.drop(['Name', 'Cabin','Combi', 'Destination'], axis=1)
# Cabin1
encode = pd.get_dummies(data['HomePlanet'])
data = data.join(encode)
encode = pd.get_dummies(data['Cabin1'])
data = data.join(encode)

data['Cabin3'] = data['Cabin3'].map({'P':0, 'S':1})
data_scaled(data, col)
dt = data.drop(['PassengerId', 'HomePlanet', 'Cabin1'], axis = 1)
dt['T'] = pd.Series(np.zeros((266))).astype('uint8')

dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CryoSleep     266 non-null    bool   
 1   Cabin2        264 non-null    float64
 2   Cabin3        264 non-null    float64
 3   Age           266 non-null    int64  
 4   VIP           266 non-null    bool   
 5   RoomService   266 non-null    float64
 6   FoodCourt     266 non-null    float64
 7   ShoppingMall  266 non-null    float64
 8   Spa           266 non-null    float64
 9   VRDeck        266 non-null    float64
 10  Earth         266 non-null    uint8  
 11  Europa        266 non-null    uint8  
 12  Mars          266 non-null    uint8  
 13  A             266 non-null    uint8  
 14  B             266 non-null    uint8  
 15  C             266 non-null    uint8  
 16  D             266 non-null    uint8  
 17  E             266 non-null    uint8  
 18  F             266 non-null    

In [107]:
X_train.columns

Index(['CryoSleep', 'Cabin2', 'Cabin3', 'Age', 'VIP', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Earth', 'Europa', 'Mars',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'T'],
      dtype='object')

In [108]:
dt.dropna(axis=0, inplace=True)

In [109]:
dt_pred = xgb_model.predict(dt)
dt_pred = pd.DataFrame(dt_pred)
dt_pred.value_counts()

0    135
2     65
1     64
dtype: int64

In [110]:
# 'TRAPPIST-1e':0, '55 Cancri e':1, 'PSO J318.5-22':2 로 변환시킨거 되돌리기
dt_pred = dt_pred.iloc[:,0].map({0:'TRAPPIST-1e', 1:'55 Cancri e', 2:'PSO J318.5-22' })
dt_pred.value_counts() # 변경 확인

TRAPPIST-1e      135
PSO J318.5-22     65
55 Cancri e       64
Name: 0, dtype: int64

In [111]:
# 예측한 값 엑셀로 내보내기
dt_pred.to_excel('Destination_fillna.xlsx')

# VIP 결측값 처리

## 데이터 불러오기

In [112]:
vip_df = spaceship.iloc[:,:]
vip_df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11076 entries, 0 to 12969
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    11076 non-null  object 
 1   CryoSleep     11076 non-null  bool   
 2   Cabin1        11076 non-null  object 
 3   Cabin2        11076 non-null  float64
 4   Cabin3        11076 non-null  bool   
 5   Destination   11076 non-null  object 
 6   Age           11076 non-null  float64
 7   VIP           11076 non-null  bool   
 8   RoomService   11076 non-null  float64
 9   FoodCourt     11076 non-null  float64
 10  ShoppingMall  11076 non-null  float64
 11  Spa           11076 non-null  float64
 12  VRDeck        11076 non-null  float64
dtypes: bool(3), float64(7), object(3)
memory usage: 1.2+ MB


## 데이터 탐색

In [113]:
vip_df.VIP.value_counts() 

False    10584
True       492
Name: VIP, dtype: int64

- 굉장히 심한 편향성을 띔

## 전처리

In [114]:
vip_df.dropna(axis=0,inplace=True)
vip_df.shape

(11076, 13)

In [115]:
vip_df.head()

,HomePlanet,CryoSleep,Cabin1,Cabin2,Cabin3,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Europa,False,B,0.0,True,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0
1,Earth,False,F,0.0,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0
2,Europa,False,A,0.0,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0
3,Europa,False,A,0.0,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0
4,Earth,False,F,1.0,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0


In [116]:
vip_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11076 entries, 0 to 12969
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    11076 non-null  object 
 1   CryoSleep     11076 non-null  bool   
 2   Cabin1        11076 non-null  object 
 3   Cabin2        11076 non-null  float64
 4   Cabin3        11076 non-null  bool   
 5   Destination   11076 non-null  object 
 6   Age           11076 non-null  float64
 7   VIP           11076 non-null  bool   
 8   RoomService   11076 non-null  float64
 9   FoodCourt     11076 non-null  float64
 10  ShoppingMall  11076 non-null  float64
 11  Spa           11076 non-null  float64
 12  VRDeck        11076 non-null  float64
dtypes: bool(3), float64(7), object(3)
memory usage: 984.3+ KB


In [117]:
# 원-핫 인코딩 (cabin1, destination)

## HomePlanet
encoding = pd.get_dummies(vip_df.HomePlanet)
vip_df = vip_df.drop('HomePlanet', axis =1) # 기존 삭제
vip_df = vip_df.join(encoding) # 적용
## Cabin1
encoding = pd.get_dummies(vip_df.Cabin1)
vip_df = vip_df.drop('Cabin1', axis =1) # 기존 삭제
vip_df = vip_df.join(encoding) # 적용

## Cabin1
encoding = pd.get_dummies(vip_df.Destination)
vip_df = vip_df.drop('Destination', axis =1) # 기존 삭제
vip_df = vip_df.join(encoding) # 적용

vip_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11076 entries, 0 to 12969
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CryoSleep      11076 non-null  bool   
 1   Cabin2         11076 non-null  float64
 2   Cabin3         11076 non-null  bool   
 3   Age            11076 non-null  float64
 4   VIP            11076 non-null  bool   
 5   RoomService    11076 non-null  float64
 6   FoodCourt      11076 non-null  float64
 7   ShoppingMall   11076 non-null  float64
 8   Spa            11076 non-null  float64
 9   VRDeck         11076 non-null  float64
 10  Earth          11076 non-null  uint8  
 11  Europa         11076 non-null  uint8  
 12  Mars           11076 non-null  uint8  
 13  A              11076 non-null  uint8  
 14  B              11076 non-null  uint8  
 15  C              11076 non-null  uint8  
 16  D              11076 non-null  uint8  
 17  E              11076 non-null  uint8  
 18  F     

In [118]:
# 스케일링을 위한 함수 생성
col = ['Cabin2', 'Age', 'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
data_scaled(vip_df, col)

,CryoSleep,Cabin2,Cabin3,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,B,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,False,-1.171058,True,0.707877,False,-0.334616,-0.282674,-0.285975,-0.271173,-0.260034,...,1,0,0,0,0,0,0,0,0,1
1,False,-1.171058,False,-0.329018,False,-0.166861,-0.277057,-0.244125,0.217775,-0.220858,...,0,0,0,0,1,0,0,0,0,1
2,False,-1.171058,False,2.021278,True,-0.268437,1.949128,-0.285975,5.709312,-0.216406,...,0,0,0,0,0,0,0,0,0,1
3,False,-1.171058,False,0.293119,False,-0.334616,0.518053,0.335083,2.693687,-0.088194,...,0,0,0,0,0,0,0,0,0,1
4,False,-1.169112,False,-0.882029,False,0.131712,-0.238987,-0.033199,0.232025,-0.258253,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12962,True,1.737845,False,0.984383,False,-0.334616,-0.282674,-0.285975,-0.271173,-0.260034,...,0,0,0,0,0,1,0,0,0,1
12963,False,-0.630138,False,0.984383,False,-0.262281,-0.282674,6.160647,-0.271173,-0.260034,...,0,0,1,0,0,0,0,0,0,1
12964,False,2.323517,False,0.777004,False,-0.334616,0.257177,-0.285975,-0.268502,-0.260034,...,0,0,0,0,1,0,0,0,0,1
12965,True,1.739790,False,0.362246,False,-0.334616,-0.282674,-0.285975,-0.271173,-0.260034,...,0,0,0,0,0,1,0,0,0,1


## 데이터와 타겟값 분리

In [119]:
vip_data = vip_df.drop(['VIP'],axis=1)
vip_label = vip_df['VIP'].replace({True:1,False:0})

In [120]:
vip_data.head()

,CryoSleep,Cabin2,Cabin3,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Earth,...,B,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,False,-1.171058,True,0.707877,-0.334616,-0.282674,-0.285975,-0.271173,-0.260034,0,...,1,0,0,0,0,0,0,0,0,1
1,False,-1.171058,False,-0.329018,-0.166861,-0.277057,-0.244125,0.217775,-0.220858,1,...,0,0,0,0,1,0,0,0,0,1
2,False,-1.171058,False,2.021278,-0.268437,1.949128,-0.285975,5.709312,-0.216406,0,...,0,0,0,0,0,0,0,0,0,1
3,False,-1.171058,False,0.293119,-0.334616,0.518053,0.335083,2.693687,-0.088194,0,...,0,0,0,0,0,0,0,0,0,1
4,False,-1.169112,False,-0.882029,0.131712,-0.238987,-0.033199,0.232025,-0.258253,1,...,0,0,0,0,1,0,0,0,0,1


In [121]:
vip_label.head()

0    0
1    0
2    1
3    0
4    0
Name: VIP, dtype: int64

## 오버샘플링

In [122]:
X_samp, y_samp = RandomOverSampler(random_state=109).fit_resample(vip_data, vip_label)

In [123]:
print(X_samp.shape)
print(y_samp.shape)

(21168, 23)
(21168,)


## train, test 데이터세트 분리

In [124]:
X_train,X_test,y_train,y_test = train_test_split(X_samp, y_samp, random_state=109)

## 모델링_XGBClassifier

In [125]:
# hyperopt 적용을 위한 설정 값 담기
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 20, 1),
                    'min_child_weight': hp.quniform('min_child_weight', 1, 2, 1),
                    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
                    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1)
               }

In [126]:
# 실행을 위한 함수 정의
from xgboost import XGBClassifier
def objective_func(search_space):
    xgb_clf = XGBClassifier(n_estimators=100, max_depth=int(search_space['max_depth']),
                            min_child_weight=int(search_space['min_child_weight']),
                            learning_rate=search_space['learning_rate'],
                            colsample_bytree=search_space['colsample_bytree'], 
                            eval_metric='logloss')
    accuracy = cross_val_score(xgb_clf, X_train, y_train, scoring='accuracy', cv=3)
    return {'loss':-1 * np.mean(accuracy), 'status': STATUS_OK}

In [127]:
# 최적의 파라미터 찾기
from sklearn.model_selection import cross_val_score
trial_val = Trials()
best = fmin(fn=objective_func,
            space=xgb_search_space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trial_val, rstate=np.random.default_rng(seed=9))
print('best:', best)

100%|████████████████████████████████████████████████| 50/50 [00:45<00:00,  1.10trial/s, best loss: -0.985575711766188]
best: {'colsample_bytree': 0.5978109120491624, 'learning_rate': 0.11202409007972883, 'max_depth': 20.0, 'min_child_weight': 1.0}


In [128]:
# hyperopt를 통해 얻은 최적의 파라미터를 이용한 XGBClassifier
xgb_model = XGBClassifier(n_estimators=100, learning_rate=round(best['learning_rate'], 5), 
                            max_depth=int(best['max_depth']), min_child_weight=int(best['min_child_weight']),
                            colsample_bytree=round(best['colsample_bytree'], 5)
                           )
xgb_model.fit(X_train, y_train)
pred= xgb_model.predict(X_test)

# 테스트 데이터 평가
print('정확도 : ', accuracy_score(y_test, pred))
print('혼돈행렬 : \n', confusion_matrix(y_test, pred))

정확도 :  0.9924414210128496
혼돈행렬 : 
 [[2620   40]
 [   0 2632]]


In [129]:
tr_pred = xgb_model.predict(X_train)
tr_proba = xgb_model.predict_proba(X_train)
t_pred= xgb_model.predict(X_test)
t_proba = xgb_model.predict_proba(X_test)

In [130]:
# 트레인 데이터 평가
pred= xgb_model.predict(X_train)
print('정확도 : ', accuracy_score(y_train, pred))
print('혼돈행렬 : \n', confusion_matrix(y_train, pred))

정확도 :  0.99949609473419
혼돈행렬 : 
 [[7916    8]
 [   0 7952]]


## 결측값 채우기

In [131]:
# VIP의 결측값만 비어있는 파일 불러오기
vip_nan_df = pd.read_excel('na_vip.xlsx')

In [132]:
# 이전 과정 반복 

vip_nan_df.drop(['PassengerId','Combi','Cabin','Name','Transported'],axis=1,inplace=True)
## Cabin3의 P(좌현)를 False으로 S(우현)를 True로 변경
vip_nan_df['Cabin3'].replace({'P': True,'S': False}, inplace=True)

## boolean으로 변환
vip_nan_df['CryoSleep'] = vip_nan_df['CryoSleep'].astype(bool)

vip_nan_df['Cabin3'] = vip_nan_df['Cabin3'].astype(bool)

## 변환 확인
vip_nan_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    296 non-null    object 
 1   CryoSleep     296 non-null    bool   
 2   Cabin1        287 non-null    object 
 3   Cabin2        287 non-null    float64
 4   Cabin3        296 non-null    bool   
 5   Destination   289 non-null    object 
 6   Age           287 non-null    float64
 7   VIP           0 non-null      float64
 8   RoomService   291 non-null    float64
 9   FoodCourt     293 non-null    float64
 10  ShoppingMall  292 non-null    float64
 11  Spa           293 non-null    float64
 12  VRDeck        294 non-null    float64
dtypes: bool(2), float64(8), object(3)
memory usage: 26.1+ KB


In [133]:
## object 타입의 데이터들 더미화
train_encoding = pd.get_dummies(vip_nan_df['HomePlanet'])
vip_nan_df=vip_nan_df.drop('HomePlanet',axis=1)
vip_nan_df = vip_nan_df.join(train_encoding)

train_encoding = pd.get_dummies(vip_nan_df['Cabin1'])
## 기존의 팀명 컬러 삭제
vip_nan_df=vip_nan_df.drop('Cabin1',axis=1)
vip_nan_df = vip_nan_df.join(train_encoding)


train_encoding = pd.get_dummies(vip_nan_df['Destination'])
## 기존의 팀명 컬러 삭제
vip_nan_df = vip_nan_df.drop('Destination',axis=1)
vip_nan_df = vip_nan_df.join(train_encoding)

col = ['Cabin2', 'Age', 'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
data_scaled(vip_nan_df, col)

,CryoSleep,Cabin2,Cabin3,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,...,B,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e
0,False,-1.167170,False,-0.953023,NaN,-0.302677,-0.030337,-0.220319,-0.286713,0.022392,...,0,0,0,0,1,0,0,1,0,0
1,False,-1.146095,False,-1.970321,NaN,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,...,0,0,0,0,0,1,0,0,0,1
2,True,-1.111608,True,0.403375,NaN,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,...,0,0,0,0,1,0,0,0,0,1
3,True,-1.092449,False,-1.020843,NaN,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,...,0,0,0,0,1,0,0,1,0,0
4,False,-1.136515,True,NaN,NaN,-0.247317,-0.165313,0.180697,-0.301808,1.115295,...,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,False,-0.550247,False,0.199915,NaN,-0.269203,0.452404,-0.363966,2.869616,0.076901,...,0,1,0,0,0,0,0,0,0,1
292,True,-0.619219,True,0.674654,NaN,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,...,0,1,0,0,0,0,0,0,0,1
293,False,NaN,True,0.606834,NaN,0.706693,-0.225656,-0.238275,-0.272413,-0.244020,...,0,0,0,0,0,0,0,0,0,1
294,False,2.432825,True,-0.546103,NaN,0.521298,-0.225656,-0.301120,-0.302602,-0.199050,...,0,0,0,0,1,0,0,0,0,1


In [134]:
vip_nan_data = vip_nan_df.drop(['VIP'],axis=1)
vip_nan_label = vip_nan_df.VIP

pred= xgb_model.predict(vip_nan_data)
pred = pd.DataFrame(pred)
pred.replace({0:False,1:True},inplace=True)
pred

,0
0,False
1,False
2,False
3,False
4,False
...,...
291,False
292,False
293,False
294,False


In [135]:
vip_nan_df.drop('VIP',axis=1,inplace=True)
vip_nan_df.insert(23,'VIP',pred)
vip_nan_df

,CryoSleep,Cabin2,Cabin3,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Earth,...,C,D,E,F,G,T,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,VIP
0,False,-1.167170,False,-0.953023,-0.302677,-0.030337,-0.220319,-0.286713,0.022392,1,...,0,0,0,1,0,0,1,0,0,False
1,False,-1.146095,False,-1.970321,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,1,...,0,0,0,0,1,0,0,0,1,False
2,True,-1.111608,True,0.403375,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,0,...,0,0,0,1,0,0,0,0,1,False
3,True,-1.092449,False,-1.020843,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,0,...,0,0,0,1,0,0,1,0,0,False
4,False,-1.136515,True,NaN,-0.247317,-0.165313,0.180697,-0.301808,1.115295,0,...,0,1,0,0,0,0,0,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291,False,-0.550247,False,0.199915,-0.269203,0.452404,-0.363966,2.869616,0.076901,0,...,1,0,0,0,0,0,0,0,1,False
292,True,-0.619219,True,0.674654,-0.302677,-0.225656,-0.363966,-0.302602,-0.250833,0,...,1,0,0,0,0,0,0,0,1,False
293,False,NaN,True,0.606834,0.706693,-0.225656,-0.238275,-0.272413,-0.244020,0,...,0,0,0,0,0,0,0,0,1,False
294,False,2.432825,True,-0.546103,0.521298,-0.225656,-0.301120,-0.302602,-0.199050,1,...,0,0,0,1,0,0,0,0,1,False


In [136]:
vip_nan_df.VIP.value_counts()

False    280
True      16
Name: VIP, dtype: int64

# Age 결측값 처리

- 중앙값인 27로 채워주었다

# 돈과 관련된 피처들 결측값 처리
- (RoomService, ShoppingMall, Spa, VRDeck, FoodCourt) 

- 중앙값인 0으로 채워주었다